In [ ]:
## Code written and developed by Alexandra Baranowski
## University of Cambridge
## Prof Alfonso Martinez Arias and Dr Naomi Moris
## Copyright 2020 Alexandra Baranowski

In [2]:
import sys
#print(sys.executable)

In [3]:
%matplotlib inline

import os
import tifffile as tiff
from scipy import ndimage
from scipy import stats
from scipy.integrate import trapz
import glob
import cv2
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import pandas as pd
import sys
from PIL import Image
import seaborn as sns
import mahotas

In [4]:
## location of the 8-bit split channels, per channel

data_locn_mask = ["/Users/Images/8bit Ch1/", 
                  "/Users/Images/8bit Ch2/", 
                 "/Users/Images/8bit Ch3/", 
                  "/Users/Images/8bit Ch4/"]

#print(data_locn_mask)

In [5]:
channels = [0,1,2,3]

#### Retrieving the images from the location and ordering them in ascending order (N.B. Make sure single digit 
# numbers are labeled 00, 01, 02 etc otherwise they will not be ordered correctly)

files_list0 = []; files_list1 = []; files_list2 = []; files_list3 = []

files_list = [files_list0, files_list1, files_list2, files_list3]

for i in range(0, len(channels)):
    files_list[i].append(glob.glob(data_locn_mask[i]+'*tif'))
    files_list[i][0].sort()


In [6]:
filenames0 = []; filenames1 = []; filenames2 = []; filenames3 = []

filenames = [filenames0, filenames1, filenames2, filenames3]

for j in range(0,len(channels)):
    for i in files_list[j][0]:
        base = os.path.basename(i)
        base2 = os.path.splitext(base)[0]
        filenames[j].append(os.path.splitext(base2)[0])

In [7]:
## Check that the list names are int the correct order - if the files were not named 00, 01 etc. they will not be

#print(files_list[0])

In [8]:
## Denoising

#% matplotlib inline 
matplotlib.rcParams.update({'figure.max_open_warning': 0})

blurred_0 = []
image_copies_0 = []
masks_0 = []
blurred_masks_0 = []
threshold_0 = []

for i, v in enumerate(files_list[0][0]):
    # read the original image
    image_0 = np.array(Image.open(files_list[0][0][i]))

    
    #Make a copy to prevent distortion of the original, bc all the transformations change the image
    image_copy = image_0.copy()
    image_copies_0.append(image_copy)
    image_copy2 = image_0.copy()

    
    #Gaussian Blur 
    blur = cv2.GaussianBlur(image_copy,(25,25),3) # change this depending on the image 
    blurred_0.append(blur)
    
    #Set initial threshold 
    ret, th0 = cv2.threshold(blur, 100, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    threshold_0.append(th0)
    mask_inv= cv2.bitwise_not(th0)
    
    
    #blurred mask of the gastruloid - for the fluorescence
    blurred_masked_img = cv2.bitwise_and(blur,blur, mask = th0)

    # bluured mask of the background - to remove bg noise 
    #blurred_bg_mask = cv2.bitwise_and(blur,blur, mask = mask_inv)      
    blurred_masks_0.append(blurred_masked_img)


In [9]:
### Histogram-based determination of the correct threshold for segmentation ###
### If the image after Gaussian blur looks bimodal, then use Otsu's ###
# Otsu's will set threshold automatically; if not, use historgrams to determine threshold to use #

# https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_thresholding/py_thresholding.html
for i, v in enumerate(blurred_0[0:6]): # choose a subset of the images or all of them
    
    ## finding the range of the grayscale values so we can do the thresholding properly 
    # https://mmeysenburg.github.io/image-processing/05-creating-histograms/ 
    hist = cv2.calcHist([v], [0], None, [256], [0, 256])
    histogram2=cv2.calcHist(image_copies_0[i], [0], None, [256], [0, 256])
    fig, ax = plt.subplots(1,2,figsize=(10,3))
    
    ax[0].plot(histogram2)
    ax[0].set_title("Grayscale Histogram") # Before de-noise
    ax[0].set_xlabel("Grayscale Value")
    ax[0].set_ylabel("Pixels")
    ax[1].plot(hist)
    ax[1].set_title("Grayscale Blurred Histogram") # After Gaussian Filter
    ax[1].set_xlabel("Grayscale Value")
    ax[1].set_ylabel("Pixels")

#plt.tight_layout()
#plt.show()

save_locn = "/Users/Image_Output/"


In [10]:
## Identify gastruloids

from scipy.interpolate import splprep, splev
im_out_ = [] # stores the initial floodfilled mask - this will suffice for some gastruloids
contours_0rough = [] # only needed if contour smoothing is applied
contours_0 = [] # stores the contours for downstream use
erosions = [] # stores the mask after erosions to remove debris surrounding gastruloids 
smoothed_0 = [] # stores the smoothed contours
count = 1;

for i, v in enumerate(image_copies_0):
    blurred_copy = v.copy()
    blurred_copy1 = v.copy()
    image_copy = v.copy()
    v2 = v.copy()
    
    # Try DoG (imgf) instead of simple Gaussian when Glds have debris around them
    blur = cv2.GaussianBlur(image_copy,(25,25),3) 
    
    # The threshold can be kept at 255 bc the Otsu's binarisation algorithm finds the optimal threshold for each image
    # this is returned as the ret value 
    ret, th01 = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    mask_inv = cv2.bitwise_not(th0)
    th0 = cv2.erode(th01, (5,5) ,iterations =1)
    
    im_floodfill_0 = th0.copy()
    
    # Mask used for flood filling; Notice the size needs to be 2 pixels larger than the image.
    h, w, = th0.shape[:2]
    mask = np.zeros((h+2, w+2), np.uint8)

    # Floodfill from point (0, 0)
    cv2.floodFill(im_floodfill_0, mask, (0,0), 255)

    # Invert floodfilled image
    im_floodfill_inv_0 = cv2.bitwise_not(im_floodfill_0)

    # Combine the two images to get the foreground
    im_out_0 = th0 | im_floodfill_inv_0
    im_out_.append(im_out_0)
    
    #### Cleaning the mask - some gastruloids will NOT require this (e.g., if they are clearly distinguishable from
    # background, have smooth edges etc)
    kernel = np.ones((25,25),np.uint8)
    erosion = cv2.erode(im_out_0, kernel ,iterations =1)
    erosions.append(erosion)
    
    # Finding the contours for each method - this is for comparison; choose the one that best delineates the gastruloids
    (contours, _) = cv2.findContours(erosion, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    (contours2, _) = cv2.findContours(erosion, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    (contours3, _) = cv2.findContours(im_out_0, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    (contours4, _) = cv2.findContours(th0, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    contours2 = sorted(contours2, key = cv2.contourArea, reverse = True)[:1]
    contours3 = sorted(contours3, key = cv2.contourArea, reverse = True)[:1]
    contours = sorted(contours, key = cv2.contourArea, reverse = True)[:1]
    
    if not contours2: 
        contours2 = np.array([0])
    contours_0rough.append(contours2)  
    
    if not contours3: 
        contours3 = np.array([0])
    smoothed_0.append(contours3)
    
    # ---- add smoothing code excerpt here, if wanted ------

    # -------------------------------------------
    
    v3 = v.copy()
    cv2.drawContours(v3, contours2, -1, (255, 0, 0), 10)     # draw contours over original image
    cv2.drawContours(v2, contours3, -1, (255, 0, 0), 10)     # draw contours over original image

    #plot it 
    fig, ax = plt.subplots(1, 6, figsize = (16,5))
    ax[0].imshow(th0)
    ax[0].set_xlabel("Distance / Pixels")
    ax[0].set_ylabel("Distance / Pixels")
    ax[0].set_title("Before Floodfilling")
    
    ax[1].imshow(im_out_0)
    ax[1].set_xlabel("Distance / Pixels")
    ax[1].set_title("After Floodfilling")
    
    ax[2].imshow(erosion) 
    ax[2].set_xlabel("Distance / Pixels")
    ax[2].set_title("After Floodfilling & Erosion ")
    
    ax[3].imshow(v)
    ax[3].set_xlabel("Distance / Pixels")
    ax[3].set_title("No Floodfill contour")
    
    ax[4].imshow(v2)
    ax[4].set_xlabel("Distance / Pixels")
    ax[4].set_title("Floodfill contour")

    ax[5].imshow(v3)
    ax[5].set_xlabel("Distance / Pixels")
    ax[5].set_title("Floodfill & Erosion contour")
    
    
    #plt.tight_layout()
    #plt.show()
    
    save_locn = "/Users/Images/"
    fig.savefig(save_locn + 'Contours_' + str(count) + '.png')
    
    count = count+1;
    
    


In [11]:
## Feature extraction

areas_0 = []
areas0 = [] # this will only be used for QC
for i, v in enumerate(erosions):
    #find areas
    filledareas = cv2.countNonZero(v)
    areas0.append(filledareas)


for i, c in enumerate(smoothed_0):
    if np.size(c)>1:
        cnt = c[0]
        area = cv2.contourArea(cnt)
        areas_0.append(area)
    else:
        areas_0.append(0)

#print(areas_0)
#print(areas0)
#print(len(areas_0))

In [12]:

matplotlib.rcParams.update({'figure.max_open_warning': 0})
perimeters_0 = []
circularity_0 = []
boxes_0 = []

In [13]:
for (i,c) in enumerate(contours_0rough): # input is the list of contours, depending on which was used above 
    
    if np.size(c)>1:
        cnnt = c[0]
        # perimeters
        perimeters = cv2.arcLength(cnnt,True)
        perimeters_0.append(perimeters)
        
        #circularity
        circularity = ((4*np.pi*areas_0[i])/perimeters**2)
        circularity_0.append(circularity)

        # fitting minimum rotated rectangle
        rect = cv2.minAreaRect(cnnt)
        box = cv2.boxPoints(rect)
        box = np.int0(box)
        boxes_0.append(box)
        
    else:
        circularity_0.append(0)
        boxes_0.append(0)
        

In [14]:
aspect_ratios_0 = []

width = []
height = []

    
## NB: boundingRect doesn't include rotation, but minAreaRect does   
for i, v in enumerate(contours_0rough):
    cnt = v[0]     
    x,z,r = cv2.minAreaRect(cnt) #outputs are center(x,y), (width, height) and rotation
    width.append(z[0])
    height.append(z[1])    

    
for i, v in enumerate(width): # normalise the AR so that all values are > 1 so the actual distance can be measured
    k = height[i]
    #print(k)
    if k > v:
        aspect_ratios_0.append(np.float(v) / k)
    else:
        aspect_ratios_0.append(np.float(k) / v)
    

#print(aspect_ratios_0)

In [15]:
print("1) Area list:", len(areas_0))
print("2) Circularity list:", len(circularity_0))
print("3) AR list:", len(aspect_ratios_0))

if len(areas_0) == len(circularity_0) == len(aspect_ratios_0) :
    print("\nCheck complete - All lists are of same length")
else: 
    print("\nError: all list lengths are not equal")

1) Area list: 0
2) Circularity list: 0
3) AR list: 0

Check complete - All lists are of same length


In [16]:
## QC based on area

new_area = []

print('Number of observations before removing outliers is %d' %len(areas_0))
#print areas_0
areas = np.array(areas_0)

mean = np.mean(areas, axis=0)
sd = np.std(areas, axis=0)
bins = 50 
new_area = [x for x in areas_0 if (x > mean - 1.5 * sd)]
new_area = [x for x in new_area if (x < mean + 1.5 * sd)]

print('Min area is %d' %min(new_area))
print('Max area is %d' %max(new_area))
print('Avg area is %d'%(sum(new_area)/len(new_area)))

circ = np.array(circularity_0)

mean = np.mean(circ, axis=0)
sd = np.std(circ, axis=0)
bins = 50 
new_circ = [x for x in circularity_0 if (x > mean -1.5 * sd)]
new_circ = [x for x in new_circ if (x < mean + 2 * sd)]


# print new_area
print('Number of observations within the threshold is %d' %len(new_area))
print('Number of observations within the threshold is %d' %len(new_circ))

# want to identify which position all the outliers are in so we can remove from all the lists 
outliers_0 = []
outliers_circ_0 = []
print("Outliers are gastruloids at the following indices, with the following areas: ")

print("Based on area: ")
for (num,item) in enumerate(areas_0):
    if item not in new_area:
        print(num, item)
        outliers_0.append(num)
print("Based on circularity: ")
for (num,item) in enumerate(circularity_0):
    if item not in new_circ:
        print(num, item)
        outliers_circ_0.append(num)


Number of observations before removing outliers is 0


/Users/nmoris/opt/anaconda3/envs/py36/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/nmoris/opt/anaconda3/envs/py36/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/nmoris/opt/anaconda3/envs/py36/lib/python3.6/site-packages/numpy/core/_methods.py:217: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/Users/nmoris/opt/anaconda3/envs/py36/lib/python3.6/site-packages/numpy/core/_methods.py:186: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/Users/nmoris/opt/anaconda3/envs/py36/lib/python3.6/site-packages/numpy/core/_methods.py:209: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


ValueError: min() arg is an empty sequence

In [17]:
total_outliers = outliers_0 + outliers_circ_0 

total_outliers = np.unique(total_outliers)
total_outliers = total_outliers[::-1]
print(total_outliers)


NameError: name 'outliers_0' is not defined

In [18]:
d = {'TreatmentConditions': 'ADD VALUES','Date': 'ADD VALUES', 'Hrs Post A': 'ADD VALUES', 
      'Cell line': 'ADD VALUES', 'Pre treatment': 'ADD VALUES', 'Treatment': 'ADD VALUES', 'Cell number': 'ADD VALUES',
     'Area Ch0': areas_0, 'Circularity': circularity_0, 'Min rot rect Ch0': boxes_0,'AR Ch0': aspect_ratios_0
}  

df = pd.DataFrame(data=d)
#df
     

In [19]:
path = '/Users/Images/'

df.to_csv(path+'NameOfFile.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/Users/Images/NameOfFile.csv'